In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import requests
import numpy as np

In [ ]:
def get_app_category(app_id):
    base_url = 'http://localhost:3000/api/apps/'
    response = requests.get(base_url+app_id)
    if response.status_code == 200:
        if "genreId" not in response.json():
            print("Not found genre id in", app_id)
            return "NOT_FOUND"
        else:
            gid = response.json()["genreId"].upper()
            print(app_id, gid)
            return gid
    else:
        print(app_id, " category cannot be found")
        return "NOT_FOUND"

In [ ]:
def update_sheets(sheetname):
    df = pd.read_excel('READ_CONTACTS.xlsx', sheet_name=sheetname)
    data = []
    for index, row in df.iterrows():
        data.append([row["Count"], row["Sentences"], row["Manually Marked"], row["uses-permission"], row["permission"]])
    updated_data = []
    app_order = 1
    description_tag = None
    for row in data:
        row[1] = str(row[1])
        if row[1].startswith("##"):
            if app_order != 1:
                description_row = [np.nan, "Description Tag", description_tag, np.nan, np.nan]
                updated_data.append(description_row)
            row[0] = "##{}".format(app_order)
            app_order += 1
            app_id = row[1].split("##")[1].strip()
            category = get_app_category(app_id)
            app_category = "CATEGORY/" + category
            category_row = [np.nan, app_category , np.nan, np.nan, np.nan]
            description_tag = row[2]
            row[2] = np.nan
            updated_data.append(row)
            updated_data.append(category_row)
        else:
            updated_data.append(row)
    return updated_data

In [ ]:
def write_to_excel(updated_data, output_file):
    def background_apply(value):
        if value.startswith("##") or value.startswith("Description Tag"):
            font_weight = 'bold'
        else:
            font_weight = 'normal'
        return 'font-weight: %s' % font_weight
    
    updated_df = pd.DataFrame(updated_data, columns = ["Count", "Sentences", "Manually Marked", "uses-permission", "permission"])
    styled = updated_df.style.applymap(background_apply, subset=['Sentences'])
    styled.to_excel(output_file, engine='openpyxl')

In [ ]:
def run(file_name, sheet_name):
    updated_data = update_sheets(file_name, sheet_name)
    write_to_excel(updated_data, "{}.xlsx".format(sheet_name))

In [ ]:
run("MUHAMMET")